# Install necessary requirements.

In [7]:
# Install necessary libraries
%pip install numpy==1.26.4 h5py spacy pandas tqdm scikit-learn scipy --quiet

import spacy

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      + meson setup C:\Users\MICHA\AppData\Local\Temp\pip-install-d2urf8jp\scipy_6192f822dc2b4bc2b968c0abb91e5377 C:\Users\MICHA\AppData\Local\Temp\pip-install-d2urf8jp\scipy_6192f822dc2b4bc2b968c0abb91e5377\.mesonpy-dt85sf1k\build -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\MICHA\AppData\Local\Temp\pip-install-d2urf8jp\scipy_6192f822dc2b4bc2b968c0abb91e5377\.mesonpy-dt85sf1k\build\meson-python-native-file.ini
      The Meson build system
      Version: 1.6.0
      Source dir: C:\Users\MICHA\AppData\Local\Temp\pip-install-d2urf8jp\scipy_6192f822dc2b4bc2b968c0abb91e5377
      Build dir: C:\Users\MICHA\AppData\Local\Temp\pip-install-d2urf8jp\scipy_6192f822dc2b4bc2b968c0abb91e5377\.mesonpy-dt85sf1k\build
      Build type: native build
      Project name: SciPy
      Project version: 1.11.1
      C compiler 

Note: you may need to restart the kernel to use updated packages.


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

# Setup for the dataset

1. Download the eb_nerd dataset locally on your computer.
2. Create the following folder: `RecSysGroup27/ebnerd_data`.
3. Add the dataset to the `ebnerd_data` folder.
The folder should contain the following datasets: `ebnerd_demo`, `ebnerd_large`, `ebnerd_small`, `ebnerd_testset`

# Training and testing the model

In [3]:
# Import necessary libraries
import torch
from NRMS import NRMS
from NRMSExtended import NRMSExtended
import spacy
from Training import train, tuneParameters, testOnWholeDataset
from Testing import runOnTestSet
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import numpy as np
import time
import pandas as pd
import Utils
from datetime import datetime
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm import tqdm
import json
import random
import scipy.stats as ss
from Dataloading import ArticlesDatasetTraining, ArticlesDatasetTest
from GitMetrics import AucScore, AccuracyScore

# Check for available device
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Install danish language model if it is not already installed
try:
    nlp = spacy.load("da_core_news_lg")
    print("spaCy model 'da_core_news_lg' is already installed.")
except:
    print("spaCy model 'da_core_news_lg' not found. Installing...")
    %python -m spacy download da_core_news_sm --quiet
    nlp = spacy.load("da_core_news_lg")

# Model parameters
dropout = 0.2
h = 16                              # Number of attention heads in multi-head self-attention

# Train parameters
dataset_name = 'ebnerd_small'   # Name of the dataset used for training and validating
k = 4                           # Number of negative samples to pair with each positive sample
batch_size = 64
weight_decay = 0.0
learning_rate = 1e-3
history_size = 30               # Number of user history entries after truncation or padding
max_title_size = 20             # Number of tokens in an article title after truncation or padding
num_epochs = 100                # Not really used (takes too long to reach number of epochs)
validate_every = 200            # How many train batches between validations
validation_batch_size = 200     # The batch size for validation
n_validation_batches = 50       # How many batches to run for each validation
max_batches = 5000              # Use this if you want to end the training early

# Choose execution type
load_model = False
save_model = True
testing = False

# Main function to run the model
def main():
    model = NRMSExtended(nlp, h=h, dropout=dropout).to(DEVICE)

    if load_model:
        model.load_state_dict(torch.load('model_best_nrms_30_20.pth', map_location=DEVICE))

    # Uncomment this if you want to tune parameters
    # tuneParameters(nlp, dataset_name, k, batch_size, num_epochs, validate_every, validation_batch_size, n_validation_batches, max_batches)

    if not testing:
        train(
            model=model,
            dataset_name=dataset_name,
            k=k,
            batch_size=batch_size,
            weight_decay=weight_decay,
            learning_rate=learning_rate,
            history_size=history_size,
            max_title_size=max_title_size,
            nlp=nlp,
            save_model=save_model,
            num_epochs=num_epochs,
            validate_every=validate_every,
            validation_batch_size=validation_batch_size,
            n_validation_batches=n_validation_batches,
            max_batches=max_batches
        )

    else:
        with torch.no_grad():
            runOnTestSet(model, history_size, max_title_size, nlp)

# Run the main function
main()


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject